In [2]:
import requests
import json
import regex as re
import time
from tqdm import tqdm

In [3]:
import sys
print(sys.version)
import requests
print(requests.__version__)

3.13.5 | packaged by Anaconda, Inc. | (main, Jun 12 2025, 16:37:03) [MSC v.1929 64 bit (AMD64)]
2.32.3


In [39]:
def Qfinder(n,y):
    url = "https://query.wikidata.org/sparql"
    
    name = n
    birth_year = y
    
    query = f"""
    SELECT ?item ?itemLabel ?birthDate ?svArticle WHERE {{
      ?item wdt:P31 wd:Q5;          # instance of human
            wdt:P569 ?birthDate.    # date of birth
      ?item (rdfs:label | skos:altLabel) "{name}"@sv.
      FILTER(YEAR(?birthDate) = {birth_year})
    
      OPTIONAL {{ ?svArticle schema:about ?item ;
                            schema:isPartOf <https://sv.wikipedia.org/> }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "sv,en". }}
    }}
    """
    
    headers = {"Accept": "application/sparql-results+json","User-Agent": "NordiskFBBot/0.5 (em8824jo-s@student.lu.se) Python/3.13.5 requests/2.32.3"}
    response = requests.get(url, params={"query": query}, headers=headers)
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(response.text)
        return None
    


In [40]:
def Qfinder_noyear(n):
    url = "https://query.wikidata.org/sparql"
    
    name = n
    
    query = f"""
    SELECT ?item ?itemLabel ?svArticle WHERE {{
      ?item wdt:P31 wd:Q5.          # instance of human
      ?item (rdfs:label | skos:altLabel) "{name}"@sv.
    
      OPTIONAL {{ ?svArticle schema:about ?item ;
                            schema:isPartOf <https://sv.wikipedia.org/> }}
      SERVICE wikibase:label {{ bd:serviceParam wikibase:language "sv,en". }}
    }}
    """
    
    headers = {"Accept": "application/sparql-results+json","User-Agent": "NordiskFBBot/0.5 (em8824jo-s@student.lu.se) Python/3.13.5 requests/2.32.3"}
    response = requests.get(url, params={"query": query}, headers=headers)

    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print(response.text)
        return None

In [107]:
with open("query.json",encoding="utf-8") as f:
    data = json.load(f)
    f.close()

wiki_data = []
for p in data:
    wiki_data.append(re.findall(r"Q\p{N}+",p['item']))


with open("wiki_data_final.json",encoding="utf-8") as f:
    data2 = json.load(f)
    f.close()

our_data = []
for d in data2:
    if len(d['sources']) > 0:
        our_data.append({"name":d['name'].replace('"','\\\"'),'year':d['birth'],'description':d['description']})
our_list = []
not_found = []
multiple_found = []
for entry in tqdm(our_data[10173:]):
    name = entry['name']
    description = entry['description']
    if len(entry['year']) < 5 and entry['year'] != "":
        year = int(entry['year'])
        res = Qfinder(name,year)
        if res != None:
            if len(res["results"]["bindings"]) == 0:
                res = Qfinder_noyear(name)
                if len(res["results"]["bindings"]) == 0:
                    idx = -1
                    idx = name.find("(")
                    if idx != -1:
                        name2 = name[:idx-1]
                        res = Qfinder(name2,year)
                    if len(res["results"]["bindings"]) == 0:
                        if idx != -1:
                            res = Qfinder_noyear(name2)
                        if len(res["results"]["bindings"]) == 0:
                            not_found.append((name,description))
                        else:
                            if len(res["results"]["bindings"]) > 1:
                                multiple_found.append((res,name,description))
                            else:
                                QId = re.findall(r"Q\p{N}+",res["results"]["bindings"][0]['item']['value'])[0]
                                our_list.append((QId,name,description))
                    else:
                        if len(res["results"]["bindings"]) > 1:
                            multiple_found.append((res,name,description))
                        else:
                            QId = re.findall(r"Q\p{N}+",res["results"]["bindings"][0]['item']['value'])[0]
                            our_list.append((QId,name,description))
                else:
                    if len(res["results"]["bindings"]) > 1:
                        multiple_found.append((res,name,description))
                    else:
                        QId = re.findall(r"Q\p{N}+",res["results"]["bindings"][0]['item']['value'])[0]
                        our_list.append((QId,name,description))
            else:
                if len(res["results"]["bindings"]) > 1:
                    multiple_found.append((res,name,description))
                else:
                    QId = re.findall(r"Q\p{N}+",res["results"]["bindings"][0]['item']['value'])[0]
                    our_list.append((QId,name,description))
        else:
            break;

[{'name': 'Publius Nigidius Figulus', 'year': '90', 'description': 'Publius Nigidius Figulus, född omkring 98 f.Kr., död omkring 45 f.Kr., var en Romarriket romersk politiker och lärd.'}]


100%|██████████| 3787/3787 [19:03<00:00,  3.31it/s]  
